In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [16]:
%matplotlib inline
from pathlib import Path
import functools
import itertools
from dataclasses import dataclass, fields

import pandas as pd
import numpy as np

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib import rcParams 

from matplottoy.data import fruit
from matplottoy.encoders import color, position
from matplottoy.artists import point, bar, utils


spath = Path("../../../draft/figures/code/")
rcParams['figure.dpi'] = 200

In [17]:
data = {'fruit': ['apple', 'orange', 'lemon', 'lime'], 
       'calories':[95, 67, 17, 20], 
       'juice':[True, True, False, False]}
df = pd.DataFrame(data)

In [18]:
data = fruit.FruitSection(df)

In [19]:
df.index.inferred_type

'integer'

In [20]:
data.local_section({'calories':(17, 20)})

FruitSection(section=   fruit  calories  juice
2  lemon        17  False
3   lime        20  False)

In [21]:
data.projection('fruit').values()

array(['apple', 'orange', 'lemon', 'lime'], dtype=object)

In [22]:
# should K be up front or last?
def constant_color(c):
     return lambda x: [mcolors.to_rgba(c) for _ in x]

def constant_value(v):
     return lambda x: [v for _ in x]

In [23]:
constant_color('k')([1,2,3,4])
constant_value(1)([1,2,3,4])

[1, 1, 1, 1]

In [24]:
V = {'height': bar.Mu(lambda x : x, 'calories'), 
     'position': bar.Mu(position.Nominal({'apple': 0, 'orange': 2, 'lemon': 4, 'lime': 6}), 'fruit'),
     'floor': bar.Mu(constant_value(0), None), 
     'width': bar.Mu(constant_value(.8), None),
     'facecolor': bar.Mu(color.Nominal({True: 'orange', False: 'blue'}), 'juice'), 
     'edgecolor':bar.Mu(constant_color('k'), None),
     'linewidth':bar.Mu(constant_value(2), None),
     'linestyle':bar.Mu(constant_value((None, None)),  None)}

In [25]:
BB = bar.Bar()

In [26]:
BB.V

{'width': Mu(nu=None, F=None),
 'position': Mu(nu=None, F=None),
 'floor': Mu(nu=None, F=None),
 'height': Mu(nu=None, F=None),
 'edgecolor': Mu(nu=None, F=None),
 'linestyle': Mu(nu=None, F=None),
 'linewidth': Mu(nu=None, F=None),
 'facecolor': Mu(nu=None, F=None)}

In [27]:
BN = BB.compose_with_nu(**V)

In [28]:
BN.V

{'width': Mu(nu=<function constant_value.<locals>.<lambda> at 0x0000023F37815CA0>, F=None),
 'position': Mu(nu=<matplottoy.encoders.position.Nominal object at 0x0000023F38423220>, F='fruit'),
 'floor': Mu(nu=<function constant_value.<locals>.<lambda> at 0x0000023F37815E50>, F=None),
 'height': Mu(nu=<function <lambda> at 0x0000023F34852F70>, F='calories'),
 'edgecolor': Mu(nu=<function constant_color.<locals>.<lambda> at 0x0000023F3842F040>, F=None),
 'linestyle': Mu(nu=<function constant_value.<locals>.<lambda> at 0x0000023F3842F160>, F=None),
 'linewidth': Mu(nu=<function constant_value.<locals>.<lambda> at 0x0000023F3842F0D0>, F=None),
 'facecolor': Mu(nu=<matplottoy.encoders.color.Nominal object at 0x0000023F384232E0>, F='juice')}

In [29]:
BB.V

{'width': Mu(nu=None, F=None),
 'position': Mu(nu=None, F=None),
 'floor': Mu(nu=None, F=None),
 'height': Mu(nu=None, F=None),
 'edgecolor': Mu(nu=None, F=None),
 'linestyle': Mu(nu=None, F=None),
 'linewidth': Mu(nu=None, F=None),
 'facecolor': Mu(nu=None, F=None)}

In [30]:
BN = BN.compose_with_nu(
    edgecolor=bar.Mu(F=None, nu=constant_color('darkgray')))

SyntaxError: unexpected EOF while parsing (<ipython-input-30-62362a2841eb>, line 2)

In [ ]:
BB.V['edgecolor']

In [ ]:
BN.V['edgecolor']

In [ ]:
BN.make_mu(data.local_section())

In [ ]:
for (k, v) in BN.V.items():
    print(k, v.F, data.local_section().projection(v.F).values())
    v.nu(data.local_section().projection(v.F).values())

In [ ]:
bar.QHatData(BN)

In [ ]:
bwd = bar.QHatData(BN).compose_with_tau(data)

In [ ]:
bwd.query()

In [ ]:
A = bar.GenericArtist(bwd)

In [ ]:
dir(bwd.graphic)

In [ ]:
A.topArt

In [ ]:
f = lambda x : np.zeros(len(x))
f(bwd.section.projection(None).values())

In [ ]:
A.draw(None)

In [ ]:
from matplottoy.artists import bar 

In [ ]:
fig, ax = plt.subplots()
ax.add_artist(A)

In [ ]:
A.draw(None)